In [2]:
import tensorflow as tf
from tensorflow import keras

In [31]:
class custom_model (tf.keras.Model):
    def train_step (self,data):
        x , y = data

        with tf.GradientTape() as tape:
            # forward pass
            y_pred = self(x,training = True)
            loss = self.compute_loss(x=x, y=y, y_pred=y_pred, training=True)
        # gradian
        training_var = self.trainable_variables
        training_gradian = tape.gradient(loss,training_var)
        # update waight
        self.optimizer.apply_gradients(zip(training_gradian,training_var))
        # update matric
        for metric in self.metrics:
            metric.update_state(y, y_pred)

        # must return a dict
        return {m.name: m.result() for m in self.metrics}

In [32]:
import numpy as np

# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = custom_model(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Just use `fit` as usual
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=3)

Epoch 1/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - mae: 0.4221 - loss: 0.4961   
Epoch 2/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - mae: 0.3992 - loss: 0.4779 
Epoch 3/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - mae: 0.3969 - loss: 0.4752 


## going lower

In [45]:
loss_tracker = keras.metrics.Mean(name='loss')
mae_metric = keras.metrics.MeanAbsoluteError(name='mae')

class custom_model_low(tf.keras.Model):
    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            # پیش‌بینی خروجی مدل با داده‌های ورودی x و حالت آموزش فعال
            y_pred = self(x, training=True)
            # محاسبه خطای میانگین مربعات بین برچسب‌ها و پیش‌بینی‌ها
            loss = tf.keras.losses.mse(y, y_pred)
            # میانگین‌گیری خطا برای به دست آوردن مقدار اسکالر
            loss = tf.reduce_mean(loss)

        # گرفتن متغیرهای قابل آموزش مدل
        training_vars = self.trainable_variables
        # محاسبه گرادیان‌ها نسبت به متغیرهای آموزش‌پذیر
        gradients = tape.gradient(loss, training_vars)
        # اعمال گرادیان‌ها برای به‌روزرسانی وزن‌ها
        self.optimizer.apply_gradients(zip(gradients, training_vars))

        # به‌روزرسانی مقدار میانگین خطا
        loss_tracker.update_state(loss)
        # به‌روزرسانی متریک میانگین مربعات خطا
        mae_metric.update_state(y, y_pred)
        # بازگرداندن دیکشنری شامل نتایج متریک‌ها
        return {"loss": loss_tracker.result(), "mae": mae_metric.result()}

    @property
    def metrics(self):
        # لیست متریک‌ها برای ریست خودکار در ابتدای هر اپوک یا ارزیابی
        return [loss_tracker, mae_metric]

In [46]:
# تعریف ورودی مدل با شکل 32
inputs = keras.Input(shape=(32,))
# تعریف لایه خروجی مدل با یک نرون
outputs = keras.layers.Dense(1)(inputs)
# ساخت نمونه‌ای از مدل سفارشی با ورودی و خروجی مشخص‌شده
model = custom_model_low(inputs, outputs)

# کامپایل مدل با استفاده از بهینه‌ساز آدام
model.compile(optimizer="adam")

# داده‌های تصادفی برای آموزش مدل
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))

# آموزش مدل به مدت 5 اپوک
model.fit(x, y, epochs=5)

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2225 - mae: 0.3775   
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2000 - mae: 0.3606 
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1843 - mae: 0.3474 
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1775 - mae: 0.3435 
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1769 - mae: 0.3407 


**## Supporting `sample_weight` & `class_weight`

If the class distribution in a dataset is not balanced—for example, 20% of the samples belong to class one and the rest to class two—then you should assign weights so that the model does not overfit to one class.


In [55]:

class CustomModel(keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        if len(data) == 3:
            x, y, sample_weight = data
        else:
            sample_weight = None
            x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value.
            # The loss function is configured in `compile()`.
            loss = self.compiled_loss(
                y,
                y_pred,
                sample_weight=sample_weight,
                regularization_losses=self.losses,
            )

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics.
        # Metrics are configured in `compile()`.
        self.compiled_metrics.update_state(y, y_pred, sample_weight=sample_weight)

        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}


In [56]:
# ساخت و کامپایل نمونه‌ای از مدل
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# داده‌های نمونه برای آموزش
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
sw = np.random.random((1000, 1))

# آموزش مدل با استفاده از sample_weight
model.fit(x, y, sample_weight=sw, epochs=3)

Epoch 1/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - mae: 0.9953 - loss: -0.4752   
Epoch 2/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - mae: 0.5847 - loss: 0.0043  
Epoch 3/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - mae: 0.4208 - loss: 0.3119  


## GAN Network

Let’s go through a complete end-to-end example step by step that uses all the concepts you’ve learned.

Key components:
- A generator network designed to produce 28×28×1 images.
- A discriminator network designed to classify 28×28×1 images into two categories (“fake” and “real”).
- An optimizer for each network.
- A loss function for training the discriminator.

In [4]:
from tensorflow import keras
from tensorflow.keras import layers

# تعریف تشخیص‌دهنده
discriminator = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),  # ورودی: تصاویر 28×28 با یک کانال (مثل تصاویر خاکستری MNIST)
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),  # لایه کانولوشنی: 64 فیلتر 3×3، گام 2 (کاهش ابعاد به نصف)، پدینگ یکسان
        layers.LeakyReLU(negative_slope=0.2),  # فعال‌سازی LeakyReLU با شیب 0.2 برای مقادیر منفی
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),  # لایه کانولوشنی دوم: 128 فیلتر 3×3، گام 2 (کاهش دوباره ابعاد)
        layers.LeakyReLU(negative_slope=0.2),  # فعال‌سازی LeakyReLU با شیب 0.2
        layers.GlobalMaxPooling2D(),  # تجمیع حداکثری برای کاهش نقشه ویژگی به یک بردار
        layers.Dense(1),  # لایه خروجی: یک نورون برای طبقه‌بندی (واقعی یا جعلی)
    ],
    name="discriminator",  # نام مدل: تشخیص‌دهنده
)

# تعریف مولد
latent_dim = 128  # بعد بردار نویز ورودی
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),  # ورودی: بردار نویز تصادفی با بعد 128
        layers.Dense(7 * 7 * 128),  # لایه متراکم: تبدیل بردار به 6272 نورون (برای نقشه ویژگی 7×7×128)
        layers.LeakyReLU(negative_slope=0.2),  # فعال‌سازی LeakyReLU با شیب 0.2
        layers.Reshape((7, 7, 128)),  # تغییر شکل به نقشه ویژگی 7×7 با 128 کانال
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),  # لایه ترانشده کانولوشنی: 128 فیلتر 4×4، گام 2 (افزایش ابعاد به 14×14)
        layers.LeakyReLU(negative_slope=0.2),  # فعال‌سازی LeakyReLU با شیب 0.2
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),  # لایه ترانشده کانولوشنی دوم: 128 فیلتر 4×4، گام 2 (افزایش ابعاد به 28×28)
        layers.LeakyReLU(negative_slope=0.2),  # فعال‌سازی LeakyReLU با شیب 0.2
        layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"),  # لایه کانولوشنی نهایی: تولید تصویر 28×28×1 با فعال‌سازی سیگموید (خروجی در بازه [0, 1])
    ],
    name="generator",  # نام مدل: مولد
)

### General Overview

- **Discriminator:**
  - Task: Examines input images (28×28×1) to determine whether they are real (from real data) or fake (generated by the generator).
  - Architecture: Uses convolutional (Conv2D) layers for downsampling and feature extraction, followed by max pooling and a dense layer for the final binary classification output.
  - Activation: LeakyReLU is used instead of ReLU to preserve small negative gradients and prevent the “dead neuron” problem.

- **Generator:**
  - Task: Generates 28×28×1 images from a random noise vector of dimension 128.
  - Architecture: The noise vector is first transformed into a 7×7×128 feature map, then upsampled through Conv2DTranspose layers to 14×14 and finally to 28×28. The final layer produces a single-channel image.
  - Activation: LeakyReLU for the intermediate layers and Sigmoid for the final output (to normalize values to [0,
This code defines a simple GAN that generates grayscale images (like MNIST) and, with proper training, can produce images resembling real data.

In [5]:
generator.summary()

Model: "generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 6272)           │       809,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_7 (LeakyReLU)       │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 14, 14, 128)    │       262,272 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_8 (LeakyReLU)       │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_3              │ (None, 28, 28, 128)    │       262,272 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_9 (LeakyReLU)       │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 1)      │         6,273 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,339,905 (5.11 MB)

 Trainable params: 1,339,905 (5.11 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
discriminator.summary()

Model: "discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 14, 14, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_6 (LeakyReLU)       │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling2d_1          │ (None, 128)            │             0 │
│ (GlobalMaxPooling2D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 74,625 (291.50 KB)

 Trainable params: 74,625 (291.50 KB)

 Non-trainable params: 0 (0.00 B)